# PEACHES Hackathon - Carbond

At this point you should have seen the short presentation on what Carbond is and know the context.

## Approach

__Goal:__ We want to measure the operational and embodied emissions of our CPU.

To achieve this we will measure the CPU package power and the CPU cycles for a number of executions of the `factorial` function.

This notebook should be run twice, once while on wall power, and once while on battery.<br/>
Copy the results of both runs and compare them. What are the differences?

## Execution

#### 1. Generate Energy/Cycles Measurements

First we need to generate energy and cycles measurements, which can then be converted to operational and attributed embodied carbon emissions.<br/>
For this, we will calculate the factorial of 100_000 150 times.

For this execute the file `measure_factorial.py` as admin.<br/>
(For security reasons (side channels), RAPL is only available for root)<br/>
Therefore, this can be done by:
```bash
cd ~/carbond-hackathon/measurements
sudo .venv/bin/python3 measurements/measure_factorial.py
```

#### 2. Import required libraries

In [110]:
import pandas as pd
from pathlib import Path

#### 3. Load the results

In [111]:
# read the csv
energy_samples = pd.read_csv("energy-samples.csv", sep=";")
# print the head of the csv
energy_samples

,timestamp,tag,duration,package_0,core_0,uncore_0
0,1.724340e+09,factorial,0.145252,2347650.0,1707515.0,42358.0
1,1.724340e+09,factorial,0.144044,2137019.0,1534664.0,31983.0
2,1.724340e+09,factorial,0.142851,2148493.0,1550777.0,29540.0
3,1.724340e+09,factorial,0.148821,2542841.0,1894893.0,34790.0
4,1.724340e+09,factorial,0.195693,3372550.0,2354608.0,144165.0


We can see a table with 6 columns.

| Name              | Description |
| ---------------- | ---------- |
| Timestamp: | the time the factorial function was started |
| Tag: | the name of the function that was executed |
| Duration: | how long it took to execute the function |
| package_0: | the µJ used by the CPU package during the execution |
| core_0: | the µJ used by all CPU cores during the execution |
| uncore_0: | the µJ used by the integrated GPU during the execution |

In [112]:
# read the csv
cpu_samples = pd.read_csv("cpu-samples.csv", sep=";")
# calculate cycles from frequency and cpu_time
cpu_samples["cycles"] = (cpu_samples["frequency"] * 1_000_000) * cpu_samples["cpu_time"]

# print the head of the csv
cpu_samples

,frequency,cpu_time,cycles
0,787.500000,0.171032,1.346881e+08
1,1667.492875,0.169514,2.826630e+08
2,1775.608250,0.169122,3.002950e+08
3,3104.191250,0.176008,5.463628e+08
4,2447.223250,0.205181,5.021240e+08


#### 4. Load the current system-intensity and CPU embodied emissions per functional unit

In [113]:
# load the system-intensity from carbond
system_intensity = Path("/var/carbond/operational/system-intensity").read_text()
system_intensity = float(system_intensity.split(" ")[0]) # remove the unit
print(f"Intensity is: {system_intensity} gCO2e/kWh")

Intensity is: 497.399 gCO2e/kWh


In [114]:
# load the CPU embodied-intensity from carbond
cpu_intensity = Path("/var/carbond/embodied/cpu").read_text()
cpu_intensity = float(cpu_intensity.split(" ")[0])  # remove the unit
print(f"Intensity is: {cpu_intensity} pg/cycle")

Intensity is: 5000.0 pg/cycle


#### 5. Calculate the average package operational emissions

In [115]:
# calculate the mean of all package_0 entries
avg_package_usage_uj = float(energy_samples["package_0"].mean())
avg_package_usage_kwh = avg_package_usage_uj / 3_600_000_000
print(f"Average package_0 energy usage is: {avg_package_usage_uj} µJ ({avg_package_usage_kwh} kWh)")


Average package_0 energy usage is: 2509710.6 µJ (0.0006971418333333334 kWh)


In [116]:
# calculate the carbon emissions
operational_avg_emissions_g = avg_package_usage_kwh * system_intensity
print(f"Operational carbon emission (CPU_PACKAGE) per function execution is: {operational_avg_emissions_g} gCO2e")

Operational carbon emission (CPU_PACKAGE) per function execution is: 0.3467576507581667 gCO2e


#### 6. Calculate the total package operational emissions

In [117]:
# calculate the sum of all package_0 entries
total_package_usage_uj = float(energy_samples["package_0"].sum())
total_package_usage_kwh = total_package_usage_uj / 3_600_000_000
print(f"Total package_0 energy usage is: {total_package_usage_uj} µJ ({total_package_usage_kwh} kWh)")


Total package_0 energy usage is: 12548553.0 µJ (0.0034857091666666666 kWh)


In [118]:
# calculate the carbon emissions
operational_total_emissions_g = total_package_usage_kwh * system_intensity
print(f"Operational carbon emission (CPU_PACKAGE) for all executions is: {operational_total_emissions_g} gCO2e")

Operational carbon emission (CPU_PACKAGE) for all executions is: 1.7337882537908333 gCO2e


#### 7. Calculate the average CPU embodied emissions

In [119]:
# calculate the mean of cpu cycles
avg_cycles = float(cpu_samples["cycles"].mean())
print(f"Average number of cycles is: {avg_cycles} cycles")


Average number of cycles is: 353226567.2358924 cycles


In [120]:
# calculate the carbon emissions
embodied_avg_emissions_pg = avg_cycles * cpu_intensity
embodied_avg_emissions_g = embodied_avg_emissions_pg / 1_000_000_000_000
print(f"Embodied carbon emission (CPU) per function execution is: {embodied_avg_emissions_g} gCO2e")

Embodied carbon emission (CPU) per function execution is: 1.7661328361794622 gCO2e


#### 8. Calculate the total CPU embodied emissions

In [121]:
# calculate the total of cpu cycles
total_cycles = float(cpu_samples["cycles"].sum())
print(f"Total number of cycles is: {total_cycles} cycles")


Total number of cycles is: 1766132836.1794622 cycles


In [122]:
# calculate the carbon emissions
embodied_total_emissions_pg = total_cycles * cpu_intensity
embodied_total_emissions_g = embodied_total_emissions_pg / 1_000_000_000_000
print(f"Embodied carbon emission (CPU) for all executions is: {embodied_total_emissions_g} gCO2e")

Embodied carbon emission (CPU) for all executions is: 8.83066418089731 gCO2e


#### 9. Add-up operational and embodied CPU emissions

##### 9.1 Average emissions

In [123]:
combined_avg_emissions_g = operational_avg_emissions_g + embodied_avg_emissions_g
print(f"Combined carbon emission (CPU) for each execution is: {combined_avg_emissions_g} gCO2e")

Combined carbon emission (CPU) for each execution is: 2.112890486937629 gCO2e


##### 9.2 Total emissions

In [124]:
combined_total_emissions_g = operational_total_emissions_g + embodied_total_emissions_g
print(f"Combined carbon emission (CPU) for all executions is: {combined_total_emissions_g} gCO2e")

Combined carbon emission (CPU) for all executions is: 10.564452434688143 gCO2e


## Results

In [125]:
import tabulate
data = [
    ["Operational", operational_avg_emissions_g, operational_total_emissions_g],
    ["Embodied", embodied_avg_emissions_g, embodied_total_emissions_g],
    ["Overall", combined_avg_emissions_g, combined_total_emissions_g]
]
table = tabulate.tabulate(data, tablefmt='html', headers=["", "Average", "Total"])
table

,Average,Total
Operational,0.346758,1.73379
Embodied,1.76613,8.83066
Overall,2.11289,10.5645
